In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


In [3]:
# Load the datasets
customers = pd.read_csv("C:\\Users\\patan Nishath khan\\Downloads\\Customers.csv")
products = pd.read_csv("C:\\Users\\patan Nishath khan\\Downloads\\Products.csv")
transactions = pd.read_csv("C:\\Users\\patan Nishath khan\\Downloads\\Transactions.csv")


In [4]:
# Merge customers and transactions datasets
transactions_with_customers = transactions.merge(customers, on="CustomerID")

# Aggregate transaction data by customer
customer_transactions = transactions_with_customers.groupby("CustomerID").agg(
    TotalSpent=("TotalValue", "sum"),
    TotalTransactions=("TransactionID", "count"),
    AvgTransactionValue=("TotalValue", "mean")
).reset_index()

# Merge aggregated data back with customers
customer_profile = customers.merge(customer_transactions, on="CustomerID", how="left").fillna(0)


In [5]:
# Encode region as numerical values
region_mapping = {region: idx for idx, region in enumerate(customer_profile["Region"].unique())}
customer_profile["RegionEncoded"] = customer_profile["Region"].map(region_mapping)

# Select features for similarity calculation
features = ["TotalSpent", "TotalTransactions", "AvgTransactionValue", "RegionEncoded"]

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_profile[features])


In [6]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(scaled_features)

# Function to get top 3 lookalikes for a given customer
def get_top_3_lookalikes(customer_id, similarity_matrix, customer_ids):
    customer_idx = customer_ids.index(customer_id)
    similarities = list(enumerate(similarity_matrix[customer_idx]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_3 = [(customer_ids[idx], score) for idx, score in similarities[1:4]]  # Exclude self
    return top_3

# Get top 3 lookalikes for the first 20 customers
customer_ids = customer_profile["CustomerID"].tolist()
lookalikes = {cust_id: get_top_3_lookalikes(cust_id, similarity_matrix, customer_ids) for cust_id in customer_ids[:20]}


In [6]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(scaled_features)

# Function to get top 3 lookalikes for a given customer
def get_top_3_lookalikes(customer_id, similarity_matrix, customer_ids):
    customer_idx = customer_ids.index(customer_id)
    similarities = list(enumerate(similarity_matrix[customer_idx]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_3 = [(customer_ids[idx], score) for idx, score in similarities[1:4]]  # Exclude self
    return top_3

# Get top 3 lookalikes for the first 20 customers
customer_ids = customer_profile["CustomerID"].tolist()
lookalikes = {cust_id: get_top_3_lookalikes(cust_id, similarity_matrix, customer_ids) for cust_id in customer_ids[:20]}


In [2]:
# Prepare results in a DataFrame
lookalike_df = pd.DataFrame({
    "CustomerID": lookalikes.keys(),
    "Lookalikes": [str(lookalikes[cust_id]) for cust_id in lookalikes.keys()]
})

# Save to a CSV file
lookalike_csv_path = "C:\\Users\\patan Nishath khan\\Downloads\\Lookalike.csv"
lookalike_df.to_csv(lookalike_csv_path, index=False)
print(f"Lookalike results saved to {lookalike_csv_path}")


NameError: name 'pd' is not defined